In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'
import sys
sys.path.append("../.venv/lib/python3.9/site-packages/")
sys.path.append("..")

In [2]:
from typing import Any, List

import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import ttest_rel, ttest_ind, norm
from sklearn.base import BaseEstimator
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error

from coordination.model.beta_coordination_blending_latent_vocalics import BetaCoordinationBlendingLatentVocalics
from coordination.model.utils.coordination_blending_latent_vocalics import LatentVocalicsDataset
from coordination.model.utils.beta_coordination_blending_latent_vocalics import BetaCoordinationLatentVocalicsDataSeries
from scripts.formatting import set_size

In [3]:
PLOTS_DIR = "/Users/paulosoares/manuscript/experimental_agenda/figures"

def save_plot(fig: Any, name: str):
    fig.savefig(f"{PLOTS_DIR}/{name}.pdf", format='pdf', bbox_inches='tight')

In [4]:
def load_datasets(advisor: str):
    with open(f"/Users/paulosoares/data/study-3_2022/datasets/{advisor}/mission1_dataset.pkl", "rb") as f:
        mission1_dataset = pickle.load(f)
    
    with open(f"/Users/paulosoares/data/study-3_2022/datasets/{advisor}/mission2_dataset.pkl", "rb") as f:
        mission2_dataset = pickle.load(f)
    
    with open(f"/Users/paulosoares/data/study-3_2022/datasets/{advisor}/all_missions_dataset.pkl", "rb") as f:
        all_missions_dataset = pickle.load(f)
    
    return mission1_dataset, mission2_dataset, all_missions_dataset

def load_model_inferences(advisor: str, ref_date: str, training_type: str):
    with open(f"/Users/paulosoares/data/study-3_2022/inferences/beta_gaussian/{advisor}/{training_type}/mission1/{ref_date}/inference_summaries.pkl", "rb") as f:
        mission1_summaries = pickle.load(f)
    
    with open(f"/Users/paulosoares/data/study-3_2022/inferences/beta_gaussian/{advisor}/{training_type}/mission2/{ref_date}/inference_summaries.pkl", "rb") as f:
        mission2_summaries = pickle.load(f)
    
    with open(f"/Users/paulosoares/data/study-3_2022/inferences/beta_gaussian/{advisor}/{training_type}/all_missions/{ref_date}/inference_summaries.pkl", "rb") as f:
        all_missions_summaries = pickle.load(f)
    
    return mission1_summaries, mission2_summaries, all_missions_summaries

def cohens_d(x1: np.ndarray, x2: np.ndarray):
    n1 = len(x1)
    n2 = len(x2)
    sp = np.sqrt(((n1 - 1) * np.var(x1) + (n2 - 1) * np.var(x2))/ (n1 + n2 - 2))
    
    return (np.mean(x1) - np.mean(x2)) / sp    

def compare_conditions(x1: np.ndarray, x2: np.ndarray, alternative: str = "greater", paired: bool = False):
    d = cohens_d(x1, x2)
    
    if paired:
        _, p_val = ttest_rel(x1, x2, alternative=alternative)
    else:
        _, p_val = ttest_ind(x1, x2, alternative=alternative)
    
    results = {
        "cohens_d": d, 
        "p_val": p_val
    }    
    
    return results

def team_process_scale_survey_score(series: BetaCoordinationLatentVocalicsDataSeries) -> float:
    return np.mean(np.array([value for value in series.team_process_surveys.values()]))

def team_satisfaction_survey_score(series: BetaCoordinationLatentVocalicsDataSeries) -> float:
    return np.mean(np.array([value for value in series.team_satisfaction_surveys.values()]))

In [7]:
# Loading all datasets
datasets = {
    "no_advisor": (load_datasets("no_advisor"), pd.read_csv("/Users/paulosoares/data/study-3_2022/prereg/no_advisor/outcome_measures.csv")),
    "human_advisor": (load_datasets("human_advisor"), pd.read_csv("/Users/paulosoares/data/study-3_2022/prereg/human_advisor/outcome_measures.csv")),
    "tomcat_advisor": (load_datasets("tomcat_advisor"), pd.read_csv("/Users/paulosoares/data/study-3_2022/prereg/tomcat_advisor/outcome_measures.csv")),
#     "all_conditions": load_datasets("all_conditions")
}

In [8]:
# All models and conditions we want to evaluate
models = [
    {"ref_date":"2022.12.02--15", "training_type":"single_execution"}
]

In [43]:
from functools import partial
AGGR_FN = np.mean #lambda x: np.mean(x[240:-120])
# AGGR_FN = np.max
# AGGR_FN = np.var
# AGGR_FN = partial(np.percentile, q=75)

# Hypothesis I: Predictive Power

Coordination is predictive of outcome measure ($M_i$). For each outcome measure, we train Bayesian linear model that tells us p($M_i$ | $C$) in a model with coordination and in a model without it. We then compute MSE and log-likelihood in a holdout set (using LOO) and report the average. We also compute how many times the model with coordination was better than the model without. We accept the hypothesis that the model with coordination is better than the one without if it is better 95% of the time.

In [44]:
class NullModel(BaseEstimator):
    
    def __init__(self):
        self.mean = None
        self.std = None
    
    def fit(self, X: np.ndarray, y: Any = None):
        self.mean = np.mean(y)
        self.std = np.std(y)
        return self
    
    def predict(self, X: np.ndarray):
        return np.ones(X.shape[0]) * self.mean
    
    def compute_log_likelihood(self, X: np.ndarray, y: np.ndarray, sample_weight=None):
        return norm(loc=self.mean, scale=self.std).logpdf(y)

class CoordinationModel(BaseEstimator):
    
    def __init__(self):
        self.reg = BayesianRidge(tol=1e-6, fit_intercept=True)
    
    def fit(self, X: np.ndarray, y: Any = None):
        self.reg.fit(X, y)
        return self
    
    def predict(self, X: np.ndarray):
        return self.reg.predict(X)
    
    def compute_log_likelihood(self, X: np.ndarray, y: np.ndarray, sample_weight=None):
        mean, std = self.reg.predict(X, return_std=True)
        return norm(loc=mean, scale=std).logpdf(y)

def execute_loo(X: np.ndarray, y: np.ndarray, plot: bool = False):
    null_model = NullModel()
    coord_model = CoordinationModel()
    null_mses = []
    null_nlls = []
    coord_mses = []
    coord_nlls = []
    
    loo = LeaveOneOut()
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        null_model.fit(X_train, y_train)
        null_mses.append(mean_squared_error(null_model.predict(X_test), y_test))
        null_nlls.append(-null_model.compute_log_likelihood(X_test, y_test)[0])   

        coord_model.fit(X_train, y_train)
        coord_mses.append(mean_squared_error(coord_model.predict(X_test), y_test))
        coord_nlls.append(-coord_model.compute_log_likelihood(X_test, y_test)[0])
        
        if plot:
            print(null_mses[-1])
            print(coord_mses[-1])
            fig = plt.figure()
            xs = np.linspace(0,1, 100)
            plt.scatter(X_train.flatten(), y_train)
            plt.scatter(X_test.flatten(), y_test, color="red")
            plt.plot(xs, null_model.predict(xs[:, np.newaxis]), label="Null")
            plt.plot(xs, coord_model.predict(xs[:, np.newaxis]), label="Coord")
            plt.legend()            
            plt.show()
    
    null_mses = np.array(null_mses)
    coord_mses = np.array(coord_mses)
    null_nlls = np.array(null_nlls)
    coord_nlls = np.array(coord_nlls)
    
    _, p_val_nll = ttest_rel(coord_nlls, null_nlls, alternative="less")
    _, p_val_mse = ttest_rel(coord_mses, null_mses, alternative="less")
    
    results = {
        "avg_mse_null": np.mean(null_mses),
        "std_mse_null": np.std(null_mses),
        "avg_nll_null": np.mean(null_nlls),
        "std_nll_null": np.std(null_nlls),
        "avg_mse_coord": np.mean(coord_mses),
        "std_mse_coord": np.std(coord_mses),
        "avg_nll_coord": np.mean(coord_nlls),
        "std_nll_coord": np.std(coord_nlls),    
        "p_coord_smaller_mse": (100.0 * np.sum(coord_mses < null_mses)) / len(coord_mses),
        "p_coord_smaller_nll": (100.0 * np.sum(coord_nlls < null_nlls)) / len(coord_nlls),
        "p_val_nll": p_val_nll,
        "p_val_mse": p_val_mse,
        "cohens_d_nll": cohens_d(coord_nlls, null_nlls),
        "cohens_d_mse": cohens_d(coord_mses, null_mses),
    }
    
    return results

In [45]:
outcome_measures_labels = ["M2", "M3", "M5_Victim", "M5_Threat", "Score"]

results_list = []
for condition in ["no_advisor", "human_advisor", "tomcat_advisor"]:
    for model in models:
        for i, mission in enumerate(["Mission 1", "Mission 2", "All Missions"]):
            dataset, outcome_df = datasets[condition]
            inferences = load_model_inferences(condition, model["ref_date"], model["training_type"])            
            X = np.array([AGGR_FN(summary.coordination_mean) for summary in inferences[i]])[:, np.newaxis]
            
            outcome_measures = np.array([outcome_df[outcome_df["Trial"] == s.uuid][outcome_measures_labels].values[0] for s in dataset[i].series]).T
            
            for j, outcome_measure in enumerate(outcome_measures):
                results_entry = model.copy()
                results_entry.update({
                    "condition": condition,
                    "mission": mission,
                    "outcome_measure": outcome_measures_labels[j]
                })
                results_entry.update(execute_loo(X, outcome_measure))
                results_list.append(results_entry)

df_h1 = pd.DataFrame.from_dict(results_list)
df_h1.head()

,ref_date,training_type,condition,mission,outcome_measure,avg_mse_null,std_mse_null,avg_nll_null,std_nll_null,avg_mse_coord,std_mse_coord,avg_nll_coord,std_nll_coord,p_coord_smaller_mse,p_coord_smaller_nll,p_val_nll,p_val_mse,cohens_d_nll,cohens_d_mse
0,2022.12.02--15,single_execution,no_advisor,Mission 1,M2,1.308524e+10,1.372646e+10,13.157261,0.763415,1.308524e+10,1.372646e+10,13.157261,0.763416,41.666667,50.000000,0.765215,0.852472,4.409180e-08,3.939352e-13
1,2022.12.02--15,single_execution,no_advisor,Mission 1,M3,3.909195e-02,6.718377e-02,0.145252,1.954838,4.079479e-02,6.915040e-02,0.002979,1.654870,58.333333,25.000000,0.090454,0.832207,-7.855712e-02,2.497778e-02
2,2022.12.02--15,single_execution,no_advisor,Mission 1,M5_Victim,2.064364e+03,1.794047e+03,5.297251,0.593605,2.335481e+03,2.444138e+03,5.242268,0.402504,75.000000,50.000000,0.243434,0.863826,-1.084192e-01,1.264611e-01
3,2022.12.02--15,single_execution,no_advisor,Mission 1,M5_Threat,5.326705e-04,6.796715e-04,-2.184805,1.197972,5.185833e-04,6.752204e-04,-2.037257,0.300315,66.666667,16.666667,0.696760,0.396252,1.689533e-01,-2.079447e-02
4,2022.12.02--15,single_execution,no_advisor,Mission 1,Score,3.086529e+04,4.236015e+04,6.769778,1.241946,3.612857e+04,4.330150e+04,6.643223,0.601163,66.666667,33.333333,0.315552,0.924711,-1.297128e-01,1.228780e-01


In [46]:
df_h1[(df_h1["p_val_mse"] <= 0.1)]

,ref_date,training_type,condition,mission,outcome_measure,avg_mse_null,std_mse_null,avg_nll_null,std_nll_null,avg_mse_coord,std_mse_coord,avg_nll_coord,std_nll_coord,p_coord_smaller_mse,p_coord_smaller_nll,p_val_nll,p_val_mse,cohens_d_nll,cohens_d_mse
8,2022.12.02--15,single_execution,no_advisor,Mission 2,M5_Threat,4.637227e-04,5.924559e-04,-2.293665,1.098506,2.613635e-04,4.795720e-04,-2.359901,0.490674,71.428571,21.428571,0.367892,0.029119,-7.785738e-02,-3.754503e-01
13,2022.12.02--15,single_execution,no_advisor,All Missions,M5_Threat,4.566406e-04,5.819133e-04,-2.381703,0.806837,3.492019e-04,6.048100e-04,-2.396334,0.503688,57.692308,19.230769,0.440935,0.073124,-2.175311e-02,-1.810341e-01
15,2022.12.02--15,single_execution,human_advisor,Mission 1,M2,2.880576e+09,4.434520e+09,12.580904,1.681954,2.880576e+09,4.434520e+09,12.580904,1.681954,75.000000,75.000000,0.747889,0.047736,4.006326e-08,-1.612067e-12
25,2022.12.02--15,single_execution,human_advisor,All Missions,M2,2.992478e+09,4.519777e+09,12.394100,1.001327,2.992478e+09,4.519777e+09,12.394100,1.001327,61.538462,61.538462,0.694292,0.047506,8.511682e-09,-3.307641e-12
39,2022.12.02--15,single_execution,tomcat_advisor,Mission 2,Score,3.970139e+04,3.519295e+04,6.768992,0.577572,1.901715e+04,2.587126e+04,6.532567,0.304359,76.923077,46.153846,0.095863,0.038947,-5.121418e-01,-6.697004e-01


# Hypothesis II: Increased coordination predicts higher outcome measures

Perform a p-test over samples from the posterior distribution of slopes and check if that is bigger than 0. Moreover, compute effect size using Cohen's d measurement.

In [53]:
def test_positive_slope(X: np.ndarray, y: np.ndarray, num_samples: int):
    reg = BayesianRidge(tol=1e-6, fit_intercept=True)
    reg.fit(X, y)
    
    mean = reg.coef_[0]
    std = np.sqrt(reg.sigma_.flatten()[0])
    slope_posterior = norm(loc=mean, scale=std)
    
    slope_samples = slope_posterior.rvs(num_samples)
    d = cohens_d(slope_samples, np.zeros(num_samples))
    _, p_val = ttest_ind(slope_samples, np.zeros(num_samples), alternative="greater")
    
    results = {
        "cohens_d": d, 
        "p_val": p_val
    }    
    
    return results

In [55]:
outcome_measures_labels = ["M2", "M3", "M5_Victim", "M5_Threat", "Score"]

results_list = []
for condition in ["no_advisor", "human_advisor", "tomcat_advisor"]:
    for model in models:
        for i, mission in enumerate(["Mission 1", "Mission 2", "All Missions"]):
            dataset, outcome_df = datasets[condition]
            inferences = load_model_inferences(condition, model["ref_date"], model["training_type"])            
            X = np.array([AGGR_FN(summary.coordination_mean) for summary in inferences[i]])[:, np.newaxis]
            
            outcome_measures = np.array([outcome_df[outcome_df["Trial"] == s.uuid][outcome_measures_labels].values[0] for s in dataset[i].series]).T
            
            for j, outcome_measure in enumerate(outcome_measures):
                results_entry = model.copy()
                results_entry.update({
                    "condition": condition,
                    "mission": mission,
                    "outcome_measure": outcome_measures_labels[j]
                })
                results_entry.update(test_positive_slope(X, outcome_measure, 10000))
                results_list.append(results_entry)

df_h2 = pd.DataFrame.from_dict(results_list)
df_h2.head()

,ref_date,training_type,condition,mission,outcome_measure,cohens_d,p_val
0,2022.12.02--15,single_execution,no_advisor,Mission 1,M2,0.020835,0.070360
1,2022.12.02--15,single_execution,no_advisor,Mission 1,M3,0.017584,0.106881
2,2022.12.02--15,single_execution,no_advisor,Mission 1,M5_Victim,0.012211,0.193971
3,2022.12.02--15,single_execution,no_advisor,Mission 1,M5_Threat,-2.155627,1.000000
4,2022.12.02--15,single_execution,no_advisor,Mission 1,Score,-1.320587,1.000000


In [56]:
df_h2[(df_h2["p_val"] <= 0.1) & (df_h2["cohens_d"] >= 0.1)]

,ref_date,training_type,condition,mission,outcome_measure,cohens_d,p_val
9,2022.12.02--15,single_execution,no_advisor,Mission 2,Score,1.928138,0.000000e+00
11,2022.12.02--15,single_execution,no_advisor,All Missions,M3,1.263987,0.000000e+00
23,2022.12.02--15,single_execution,human_advisor,Mission 2,M5_Threat,0.293734,4.125006e-95
28,2022.12.02--15,single_execution,human_advisor,All Missions,M5_Threat,0.113446,5.510226e-16
37,2022.12.02--15,single_execution,tomcat_advisor,Mission 2,M5_Victim,3.689654,0.000000e+00
39,2022.12.02--15,single_execution,tomcat_advisor,Mission 2,Score,5.412032,0.000000e+00
42,2022.12.02--15,single_execution,tomcat_advisor,All Missions,M5_Victim,3.401402,0.000000e+00
44,2022.12.02--15,single_execution,tomcat_advisor,All Missions,Score,3.621441,0.000000e+00


# Hypothesis III: Intervening on team communication predicts higher coordination and outcome measures


Compare mission by mission whether ToMCAT trials had higher coordination and outcome measures than others.

In [57]:
outcome_measures_labels = ["M2", "M3", "M5_Victim", "M5_Threat", "Score"]

# Compare the second group against the first
groups = [
    ("no_advisor", "human_advisor"),
    ("no_advisor", "tomcat_advisor"),
    ("human_advisor", "tomcat_advisor"),
]

results_list = []
for model in models:
    for i, mission in enumerate(["Mission 1", "Mission 2", "All Missions"]):
        for condition1, condition2 in groups:
            inferences1 = load_model_inferences(condition1, model["ref_date"], model["training_type"])            
            coord1 = np.array([AGGR_FN(summary.coordination_mean) for summary in inferences1[i]])

            inferences2 = load_model_inferences(condition2, model["ref_date"], model["training_type"])            
            coord2 = np.array([AGGR_FN(summary.coordination_mean) for summary in inferences2[i]])
            
            # Comparing coordination. Outcome measures were compared in another notebook.
            results_entry = model.copy()
            results_entry.update({
                "condition1": condition1,
                "condition2": condition2,
                "mission": mission
            })
            results_entry.update(compare_conditions(coord2, coord1))
            results_list.append(results_entry)

df_h3 = pd.DataFrame.from_dict(results_list)
df_h3.head()

,ref_date,training_type,condition1,condition2,mission,cohens_d,p_val
0,2022.12.02--15,single_execution,no_advisor,human_advisor,Mission 1,0.307606,0.239131
1,2022.12.02--15,single_execution,no_advisor,tomcat_advisor,Mission 1,-0.549728,0.894645
2,2022.12.02--15,single_execution,human_advisor,tomcat_advisor,Mission 1,-0.826947,0.967306
3,2022.12.02--15,single_execution,no_advisor,human_advisor,Mission 2,0.248999,0.265545
4,2022.12.02--15,single_execution,no_advisor,tomcat_advisor,Mission 2,0.551135,0.090475


In [58]:
df_h3[(df_h3["p_val"] <= 0.1)]

,ref_date,training_type,condition1,condition2,mission,cohens_d,p_val
4,2022.12.02--15,single_execution,no_advisor,tomcat_advisor,Mission 2,0.551135,0.090475


## Hypothesis IV: Coordination in the second mission is higher than in the first mission

In [72]:
def get_paired_coordination(mission1_dataset: LatentVocalicsDataset, mission2_dataset: LatentVocalicsDataset, 
                            mission1_coordination: np.ndarray, mission2_coordination: np.ndarray):
    """
    Gets coordination for the same teams in mission 1 and 2. Ignores data from
    trials with only one mission.
    """
    
    mission1 = {}
    mission2 = {}
    mission1_trial_numbers = []
    
    for i in range(mission1_dataset.num_trials):
        mission_trial_number = int(mission1_dataset.series[i].uuid[1:])
        mission1[mission_trial_number] = mission1_coordination[i]
        
    for i in range(mission2_dataset.num_trials):
        mission_trial_number = int(mission2_dataset.series[i].uuid[1:])
        if mission_trial_number - 1 in mission1:
            # Only add if exists an entry for mission 1
            mission1_trial_numbers.append(mission_trial_number - 1)
        
            mission2[mission_trial_number] = mission2_coordination[i]
    
    coord1 = []
    coord2 = []
    for mission1_trial_number in mission1_trial_numbers:
        coord1.append(mission1[mission1_trial_number])
        coord2.append(mission2[mission1_trial_number + 1])
    
    return np.array(coord1), np.array(coord2)

In [73]:
results_list = []
for model in models:    
    for condition in ["no_advisor", "human_advisor", "tomcat_advisor", "all_conditions"]:
        inferences = load_model_inferences(condition, model["ref_date"], model["training_type"])            
        
        dataset1 = datasets[condition][0]
        dataset2 = datasets[condition][1]
        coord1 = np.array([AGGR_FN(summary.coordination_mean) for summary in inferences[0]])          
        coord2 = np.array([AGGR_FN(summary.coordination_mean) for summary in inferences[1]])        
        
        coord1, coord2 = get_paired_coordination(dataset1, dataset2, coord1, coord2)
        
        # Comparing coordination. Outcome measures were compared in another notebook.
        results_entry = model.copy()
        results_entry.update({
            "condition": condition
        })
        
        results_entry.update(compare_conditions(coord2, coord1, paired=True))
        results_list.append(results_entry)

df_h4 = pd.DataFrame.from_dict(results_list)
df_h4.head()

,ref_date,training_type,condition,cohens_d,p_val
0,2022.12.02--15,single_execution,no_advisor,-0.600152,0.957793
1,2022.12.02--15,single_execution,human_advisor,-0.876517,0.996158
2,2022.12.02--15,single_execution,tomcat_advisor,0.689078,0.051727
3,2022.12.02--15,single_execution,all_conditions,-0.044276,0.574790
4,2022.12.04--12,single_execution_no_self_dep,no_advisor,-0.990994,0.985384


In [74]:
df_h4[(df_h4["p_val"] <= 0.05)]

,ref_date,training_type,condition,cohens_d,p_val
6,2022.12.04--12,single_execution_no_self_dep,tomcat_advisor,0.853623,0.037096
14,2022.12.05--09,single_execution_pitch_only,tomcat_advisor,1.026324,0.027777
